## 一、数据集预处理

### 1、将训练集的mentions转换为text对应的标记

In [ ]:
import pandas as pd

# 读取训练集
df = pd.read_excel("data/train.xlsx")

# entity的起始和后续标记
B_ENT = 1
I_ENT = 2

# 逐个样本处理
for index, row in df.iterrows():

    # mentions的列表
    mention_list = eval(row['mentions'])

    # text字符串的长度
    text_len = len(row['text'])

    # 创建与text等长的label
    label = [0]*text_len

    # 对mention列表的每一个mention做处理，其中每个mention是一个dict
    for mention in mention_list:

        # mention字典中的'offset'字段表示某个实体的开始和结束位置，左闭右开
        (start, end) = eval(mention['offset'])

        # entity的起始位置标记为B_ENT
        label[start] = B_ENT
        
        # entity的其余位置标记为I_ENT
        for num in range(start+1, end):
            label[num] = I_ENT

    # 存储label
    df.loc[index, "labels"] = str(label)
    
# 保存为新的CSV文件
df.to_csv("data/train_labeled.csv")


### 2、分句

#### 方案一：用句号划分

In [ ]:
df0 = pd.read_csv('data/train_labeled.csv')

# 以句号为分隔
char_sep = '。'
li = []

for index, row in df0.iterrows():
    # indices是分隔的位置list，第一句从头开始
    indices = [-1]

    text = row['text']
    text_len = len(text)

    # 找到所有的句号
    for i in range(len(text)):
        if text[i] == char_sep:
            indices.append(i)

    # 如果结尾不是句号，添加最后一个字符为分隔
    if indices[-1] != text_len - 1:
        indices.append(text_len - 1)

    # 逐句处理
    for j in range(len(indices)-1):
        begin = indices[j] + 1
        end = indices[j+1] + 1
        sent = text[begin: end]
        label = eval(row['labels'])[begin: end]
        length = end - begin
        li.append([length, sent, label])
    
df = pd.DataFrame(li, columns = ['length', 'text', 'labels'])

df.to_csv('data/sep_by_。.csv')


#### 方案二、指定最大长度

In [ ]:
df = pd.read_csv('data/train_labeled.csv')
df_s = pd.DataFrame(columns = ['segment_number','id','segment_len','text','mentions','labels'])
max_size = 100
#句段长度上限
#print(df_s.shape)
#header = 0 把第一行作为列名称，首位空缺设置为默认的0
#print(df.loc[0])
#显示列标
#预期效果：每行是一个短于max_size样本片段，一个样本占有若干行（若干片段）。
#预期效果：每列是一个片段的信息：
#         列标：0,               1，        2，      3，      4，          5，          
#              所属样本第几个片段 所属样本id 片段长度 片段文本  片段mentions  片段labels
seg_num = 0
#总片段下标

for row in df.values:
    #print(type(row),row.shape)
    mention_list = eval(row[3])
    #print(eval(row[5]))
    #al_len = 0
    al_seg_len = 0
    segs = 0
    #该文本已经切割的片段总长度和片段数目
    
    df_s.loc[seg_num] = [segs,row[1],0,'',[],'']
    #df_s = df_s.append(pd.DataFrame([[segs,row[1],0,'',[],'']],columns = df_s.columns))
    #print(type(df_s.loc[seg_num]))
    #初始化该文本第一个片段
    for i in range(len(mention_list)):
    # 对mention列表的每一个mention做处理，其中每个mention是一个dict
        
        start, end = eval(mention_list[i]['offset'])
        # mention字典中的'offset'字段表示某个实体的开始和结束位置，左闭右开
        if end < al_seg_len + max_size:
        #实体全部在长度范围内
            df_s.iloc[seg_num,4].append(mention_list[i])
            #print(mention_list[i])
            if i == len(mention_list):
            #最后一个mention
                df_s.iloc[seg_num,3]=row[2][al_seg_len:min(al_seg_len + max_size,len(row[2]))]
                df_s.iloc[seg_num,5]=str(eval(row[5])[al_seg_len:min(al_seg_len + max_size,len(eval(row[5])))])
                al_seg_len = min(al_seg_len + max_size,len(row[2]))

                df_s.iloc[seg_num,2]=len(df_s.iloc[seg_num,3])
                seg_num +=1   
                segs += 1
        else:
        #过长，该实体放入下一个片段,开始整理已经确定的片段
            i -= 1
            #下一次还要放入该mention
            df_s.iloc[seg_num,3]=row[2][al_seg_len:min(al_seg_len + max_size,start)]
            df_s.iloc[seg_num,5]=str(eval(row[5])[al_seg_len:min(al_seg_len + max_size,start)])
            
            al_seg_len = min(al_seg_len + max_size,start)

            df_s.iloc[seg_num,2]=len(df_s.iloc[seg_num,3])
            seg_num +=1   
            segs += 1
            if al_seg_len < len(row[2]): df_s.loc[seg_num] = [segs,row[1],0,'',[],'']
            #新片段
    #最后处理尾部无mentions区域
    while al_seg_len < len(row[2]):
        df_s.iloc[seg_num,3]=row[2][al_seg_len:min(al_seg_len + max_size,len(row[2]))]
        df_s.iloc[seg_num,5]=str(eval(row[5])[al_seg_len:min(al_seg_len + max_size,len(eval(row[5])))])
        al_seg_len = min(al_seg_len + max_size,len(row[2]))
        if len(eval(df_s.iloc[seg_num,5])) != len(df_s.iloc[seg_num,3]) or len(eval(df_s.iloc[seg_num,5]))>max_size: 
           print(len(df_s.iloc[seg_num,3]),len(eval(df_s.iloc[seg_num,5])))
        df_s.iloc[seg_num,2]=len(df_s.iloc[seg_num,3])
        seg_num +=1   
        segs += 1
        if al_seg_len < len(row[2]): df_s.loc[seg_num] = [segs,row[1],0,'',[],'']
    #print(df_s.loc[0])

df_s.to_csv("data/train_separated.csv")

## 二、数据处理

### 1、加载分词器并测试

In [ ]:
from transformers import AutoTokenizer

#加载分词器
tokenizer = AutoTokenizer.from_pretrained('hfl/rbt6')

# 测试文本
text = ["红土创新基金管理有限公司6月30日发布公告，红土创新盐田港REIT(180301)的基金经理新聘陈超。"
,"金能科技: 金能科技股份有限公司关于“金能转债”转股价格调整的提示性公告"]

# 将文本进行逐字分解
def split_sent(text):
    return [char for char in text]

def split(text_list):

    text_split = []
    for sent in text_list:
        sent_split = split_sent(sent)
        text_split.append(sent_split)
    return text_split


# 分解并编码
def split_and_encode(text_list):

    inputs = tokenizer.batch_encode_plus(
    split(text_list),
    truncation=True,
    padding=True,
    return_tensors='pt',
    is_split_into_words=True)

    return inputs

# 展示编码结果
inputs_test = split_and_encode(text)
for key,value in inputs_test.items():
    print(key ,": \n", value, "\n")


### 2、定义新的Dataset类

In [ ]:
import torch
import datasets

# Dataset类，用于处理训练集
class Dataset(torch.utils.data.Dataset):

    def __init__(self, path):
        
        # 用pandas读取csv，并将其转换为Dataset
        data_df = pd.read_csv(path)
        data_list = data_df.to_dict(orient="list")
        dataset = datasets.Dataset.from_dict(data_list)

        #过滤掉太长的句子
        def f(data):
            return data['length'] <= 200 - 2

        dataset = dataset.filter(f)

        self.data = dataset

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        token = split_sent(self.data[index]['text'])
        label = eval(self.data[index]['labels'])

        return token, label 

# 创建数据集实例
dataset = Dataset('data/sep_by_。.csv')

# 使用数据集
token, label = dataset[0]

len(dataset), token, label

### 3、定义数据整理函数

In [ ]:
#数据整理函数
def collate_fn(data):
    
    # tokens是分解后的文本
    tokens = [row[0] for row in data]
    # labels是对应的标记
    labels = [row[1] for row in data]

    # inputs是tokens的编码
    inputs = tokenizer.batch_encode_plus(tokens,
                                         truncation=True,
                                         padding=True,
                                         return_tensors='pt',
                                         is_split_into_words=True)

    # 编码的长度（最长长度）
    length = inputs['input_ids'].shape[1]

    # 将[CLS]和[PAD]标记为3，并将标记长度与编码长度统一
    for i in range(len(labels)):
        labels[i] = [3] + labels[i]
        labels[i] += [3] * length
        labels[i] = labels[i][:length]

    return inputs, torch.LongTensor(labels)


### 4、定义数据加载器

In [ ]:
batch_size = 20

#数据加载器
loader = torch.utils.data.DataLoader(dataset=dataset,
                                     batch_size=batch_size,
                                     collate_fn=collate_fn,
                                     shuffle=True,
                                     drop_last=True)

#查看数据样例
for i, (inputs, labels) in enumerate(loader):
    break

print(len(loader))
print(len(labels))
print(tokenizer.decode(inputs['input_ids'][0]))
print(labels)

for k, v in inputs.items():
    print(k,'\t', v.shape)

## 三、模型构建

### 1、加载预训练模型

In [ ]:
from transformers import AutoModel

# 加载预训练模型
pretrained = AutoModel.from_pretrained('hfl/rbt6')

# 统计参数量
print(sum(i.numel() for i in pretrained.parameters()) / 10000, '万')

#模型试算
pretrained(**inputs).last_hidden_state.shape

torch.save(pretrained, 'model/pretrained.model')

### 2、定义下游主体模型

In [ ]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")  # 使用可用的第一个 GPU
device1 = torch.device("cpu")

# pretrained = torch.load('model/pretrained.model')

#定义下游模型
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.tuneing = False
        self.pretrained = None

        self.rnn = torch.nn.GRU(768, 768,batch_first=True)
        self.fc = torch.nn.Linear(768, 8)

    def forward(self, inputs):
        if self.tuneing:
            out = self.pretrained(**inputs).last_hidden_state
        else:
            with torch.no_grad():
                out = pretrained(**inputs).last_hidden_state

        out, _ = self.rnn(out)

        out = self.fc(out).softmax(dim=2)

        return out

    def fine_tuneing(self, tuneing):
        self.tuneing = tuneing
        if tuneing:
            for i in pretrained.parameters():
                i.requires_grad = True

            pretrained.train()
            self.pretrained = pretrained
        else:
            for i in pretrained.parameters():
                i.requires_grad_(False)

            pretrained.eval()
            self.pretrained = None

pretrained = pretrained.to(device)
model = Model()
inputs = inputs.to(device)
model.to(device)



model(inputs).shape

In [ ]:
#对计算结果和label变形,并且移除pad
def reshape_and_remove_pad(outs, labels, attention_mask):
    #变形,便于计算loss
    #[b, lens, 8] -> [b*lens, 8]
    outs = outs.reshape(-1, 8)
    #[b, lens] -> [b*lens]
    labels = labels.reshape(-1)

    #忽略对pad的计算结果
    #[b, lens] -> [b*lens - pad]
    select = attention_mask.reshape(-1) == 1
    outs = outs[select]
    labels = labels[select]

    return outs, labels


reshape_and_remove_pad(torch.randn(2, 3, 8), torch.ones(2, 3),
                       torch.ones(2, 3))

In [ ]:
#获取正确数量和总数
def get_correct_and_total_count(labels, outs):
    #[b*lens, 8] -> [b*lens]
    outs = outs.argmax(dim=1)
    correct = (outs == labels).sum().item()
    total = len(labels)

    #计算除了0以外元素的正确率,因为0太多了,包括的话,正确率很容易虚高
    select = labels != 0
    outs = outs[select]
    labels = labels[select]
    correct_content = (outs == labels).sum().item()
    total_content = len(labels)

    return correct, total, correct_content, total_content


get_correct_and_total_count(torch.ones(16), torch.randn(16, 8))

In [ ]:
# li = []
# for step, (inputs, labels) in enumerate(loader):
#     li.append((step, inputs, labels))
# torch.save(li, 'tensor_file.pt')

In [ ]:
# loaded_tensor = torch.load('tensor_file.pt')


# print(len(loaded_tensor[0]))

In [ ]:
from torch.optim import AdamW


#训练
def train(epochs):
    lr = 2e-5 if model.tuneing else 5e-4

    #训练
    optimizer = AdamW(model.parameters(), lr=lr)
    criterion = torch.nn.CrossEntropyLoss()

    model.train()
    for epoch in range(epochs):
        for step, (inputs, labels) in enumerate(loader):
            #模型计算
            #[b, lens] -> [b, lens, 8]
            inputs = inputs.to(device)
            outs = model(inputs)

            labels = labels.to(device)

            #对outs和label变形,并且移除pad
            #outs -> [b, lens, 8] -> [c, 8]
            #labels -> [b, lens] -> [c]
            outs, labels = reshape_and_remove_pad(outs, labels,
                                                  inputs['attention_mask'])

            #梯度下降
            loss = criterion(outs, labels)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            if step % 50 == 0:
                counts = get_correct_and_total_count(labels, outs)

                accuracy = counts[0] / counts[1]
                accuracy_content = counts[2] / counts[3]

                print(epoch, step, loss.item(), accuracy, accuracy_content)

        torch.save(model, 'model/model.model')

model.fine_tuneing(False)
print(sum(p.numel() for p in model.parameters()) / 10000)
train(10)

In [ ]:
model.fine_tuneing(True)
print(sum(p.numel() for p in model.parameters()) / 10000)
train(20)

In [ ]:
train(100)

In [ ]:
device1 = torch.device("cpu")

#测试
def test():
    model_load = torch.load('model/model.model')
    model_load.eval()

    loader_test = torch.utils.data.DataLoader(dataset=dataset,
                                              batch_size=128,
                                              collate_fn=collate_fn,
                                              shuffle=True,
                                              drop_last=True)

    correct = 0
    total = 0

    correct_content = 0
    total_content = 0

    for step, (inputs, labels) in enumerate(loader_test):
        if step == 5:
            break
        print(step)

        with torch.no_grad():
            #[b, lens] -> [b, lens, 8] -> [b, lens]
            inputs = inputs.to(device1)
            model_load = model_load.to(device1)
            outs = model_load(inputs)

        #对outs和label变形,并且移除pad
        #outs -> [b, lens, 8] -> [c, 8]
        #labels -> [b, lens] -> [c]
        outs, labels = reshape_and_remove_pad(outs, labels,
                                              inputs['attention_mask'])

        counts = get_correct_and_total_count(labels, outs)
        correct += counts[0]
        total += counts[1]
        correct_content += counts[2]
        total_content += counts[3]

    print(correct / total, correct_content / total_content)


test()

In [ ]:
import torch
def predict_one():
    model_load = torch.load('model/model.model')
    model_load.eval()

    text = ["红土创新基金管理有限公司6月30日发布公告，红土创新盐田港REIT(180301)的基金经理新聘陈超。"
    ,"金能科技: 金能科技股份有限公司关于“金能转债”转股价格调整的提示性公告",
    "大地熊成功收购技研株式会社安徽大地熊新材料股份有限公司2023年6月22日,大地熊日本株式会社与技研株式会社(英文名:p.m.giken inc,简称pm公司)原有股东和相关方签订收购合同》,大地熊日本株式会社全资收购pm公司并即时启动相关手续交接。 pm公司是专业从事磁钢整体磁气回路的解析与设计,注塑磁体的开发、制造与销售。此次收购是大地熊向注塑磁体领域的全新产业布局,将进一步推进大地熊海内外战略合作与业务拓展,丰富公司磁性材料及部件产品种类,提升公司综合实力和整体竞争力。",
    "秋田满满、小鹿蓝蓝食安问题频发,投诉也没用随着我国新生代父母的科学喂养观念加强,我国婴童辅食行业消费规模持续上升。早在2019年我国婴幼儿辅食消费市场规模就已经达到404亿元,年复合增长率高达23%,预计未来我国婴幼儿辅食市场规模应在千亿以上。"]
    
    text_split = []
    for sent in text:
        sent_split = [char for char in sent]
        text_split.append(sent_split)

    #print(text_split)
    inputs = tokenizer.batch_encode_plus(
    text_split,
    truncation=True,
    padding=True,
    return_tensors='pt',
    is_split_into_words=True)


    with torch.no_grad():
        #[b, lens] -> [b, lens, 8] -> [b, lens]
        outs = model_load(inputs).argmax(dim=2)

    for i in range(len(text)):
        #移除pad
        select = inputs['attention_mask'][i] == 1
        input_id = inputs['input_ids'][i, select]
        out = outs[i, select]
        #label = labels[i, select]
        
        #输出原句子
        print(tokenizer.decode(input_id).replace(' ', ''), len(input_id))

        #输出tag
        for tag in [out]:
            s = ''
            for j in range(len(tag)):
                if tag[j] == 0:
                    s += '·'
                    continue
                s += tokenizer.decode(input_id[j])
                s += str(tag[j].item())

            print(s)
        print('==========================')
    
predict_one()

In [ ]:
import torch
print(torch.cuda.is_available())